In [1]:
# '''
# !pip3 install z3-solver
# !pip3 install utils
# !pip3 install gurobipy
# '''

In [2]:
from os import scandir
import csv
import numpy as np
from math import log2
import z3
from z3 import *
import time

In [3]:
filespath = 'Instances/'
data_list = []

for filename in os.listdir(filespath):
    with open(os.path.join(filespath, filename)) as f:
        lines = f.readlines()

        num_couriers = int(lines[0].strip())
        num_items = int(lines[1].strip())
        courier_loads = [int(i) for i in lines[2].strip().split()]
        item_sizes = [int(i) for i in lines[3].strip().split()]
        distance_matrix = [[int(j) for j in i.strip().split()] for i in lines[4:]]

        data_list.append({
            "m": num_couriers,
            "n": num_items,
            "l": courier_loads,
            "s": item_sizes,
            "D": distance_matrix
        })
data_list = sorted(data_list, key=lambda d: d['m'] + d['n'])
for i in range(20):
    data = data_list[i]
    m = data['m']  # num_couriers
    n = data['n']  # num_items

In [4]:
def define_decision_variables(num_couriers, num_items):
    #These variables control the couriers’ movements and ensure that they follow the optimal path
    goes_to = [[Bool(f"x_{i}_{j}") for j in range(num_items+1)] for i in range(num_items+1)]
    #These variables signal ’true’ when courier k is allocated to node i
    courier_assignment = [[Bool(f"v_{i}_{k}") for k in range(num_couriers)] for i in range(num_items)]

    num_bits = int(log2(num_items))+1
    #These variables purpose is to encode the sequence of node visits in binary notation
    u = [[Bool(f"u_{i}_{k}") for k in range(num_bits)] for i in range(num_items)]
    return goes_to, courier_assignment, u

In [5]:
def exactly_one_np(boolean_vars):
    return And(AtMost(*boolean_vars, 1), AtLeast(*boolean_vars, 1))

In [6]:
def exactly_one_he(bool_vars):
    n = len(bool_vars)
    clauses = []
    # Ensure at least one variable is True
    clauses.append(Or(bool_vars))

    # Ensure at most one variable is True
    for i in range(n):
        for j in range(i + 1, n):
            clauses.append(Or(Not(bool_vars[i]), Not(bool_vars[j])))

    return And(clauses)

In [ ]:
def at_least_one_he(bool_vars):
    clauses = []
    n = len(bool_vars)

    # Create auxiliary variables
    aux_vars = [Bool(f"aux_{i}") for i in range(n - 1)]

    # Encode the "at least one" constraint using Heule encoding
    clauses.append(Or(bool_vars[0], aux_vars[0]))
    clauses.append(Or(Not(bool_vars[0]), Not(aux_vars[0])))

    for i in range(1, n - 1):
        clauses.append(Or(bool_vars[i], aux_vars[i], Not(aux_vars[i - 1])))
        clauses.append(Or(Not(bool_vars[i]), aux_vars[i], Not(aux_vars[i - 1])))
        clauses.append(Or(Not(bool_vars[i]), Not(aux_vars[i]), aux_vars[i - 1]))

    clauses.append(Or(bool_vars[-1], Not(aux_vars[-1])))
    clauses.append(Or(Not(bool_vars[-1]), aux_vars[-1]))

    return And(clauses)


In [7]:
exactly_one = exactly_one_he

In [8]:
def add_constraints(solver, goes_to, courier_assignment, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance):
    # No route from itself to itself
    solver.add([goes_to[i][i] == False for i in range(num_items+1)])

    # Each location must be visited exactly once
    for i in range(num_items):
        solver.add(exactly_one([goes_to[i][j] for j in range(num_items+1)]))
        solver.add(exactly_one([goes_to[j][i] for j in range(num_items+1)]))

    # At most num_couriers couriers can start from the origin and return to the origin
    solver.add((Sum([goes_to[num_items][j] for j in range(num_items)])== num_couriers))
    solver.add((Sum([goes_to[i][num_items] for i in range(num_items)])== num_couriers))

    solver.add([[goes_to[j][i] == False for i in range(num_items)] for j in range(num_items) if i < j and distance_matrix[i,j] == distance_matrix[j,i] ])

    # If courier i goes from location j to location k, then courier i must also carry item j
    solver.add([Implies(goes_to[i][j], courier_assignment[i][k] == courier_assignment[j][k]) for k in range(num_couriers) for i in range(num_items) for j in range(num_items)])

    # Each item must be carried by exactly one courier
    for i in range(num_items):
        solver.add(exactly_one([courier_assignment[i][k] for k in range(num_couriers)]))

    # The total size of items carried by each courier must not exceed the courier's max load
    # PbLe to restrict the weighted sum of each items of each couriers to be less or equal to their max_loads
    for k in range(num_couriers):
        solver.add(PbLe([(courier_assignment[i][k], item_sizes[i]) for i in range(num_items)], max_loads[k]))

    # The total distance travelled by the couriers must not exceed the max distance
    solver.add(PbLe([(goes_to[i][j], distance_matrix[i, j]) for j in range(num_items+1) for i in range(num_items+1) if i!=j], max_distance-1))

    # Symmetry-breaking constraint: If courier i goes from the origin to location j, then i must be less than j, order of node assignment
    solver.add([Implies(And(goes_to[num_items][i],goes_to[i][j]),i<j) for i in range(num_items) for j in range(num_items) if i!=j])

    # Each courier must carry at least one item
    solver.add([AtLeast(*[courier_assignment[i][k] for i in range(num_items)], 1) for k in range(num_couriers)])


In [9]:
def define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance=None):
    distance_matrix = np.array(distance_matrix)

    if max_distance is None:
        max_distance = int(np.sum(distance_matrix))

    solver = Optimize()

    goes_to, courier_assignment, u = define_decision_variables(num_couriers, num_items)

    add_constraints(solver, goes_to, courier_assignment, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance)

    # New variable to represent the maximum distance travelled by any courier
    max_courier_distance = Int("max_courier_distance")

    # Constraint to update the maximum distance based on the courier distances
    courier_distances = [Sum([distance_matrix[i, k] * goes_to[i][k] for k in range(num_items + 1) if k != i]) for i in range(num_items + 1)]
    solver.add(max_courier_distance == If(len(courier_distances) > 0, maximum(courier_distances), 0))

    # Minimize the maximum distance travelled by any courier
    solver.minimize(max_courier_distance)
    
    return solver, goes_to, courier_assignment, max_courier_distance

In [10]:
def maximum(lst):
    max_val = lst[0]
    for val in lst[1:]:
        max_val = If(val > max_val, val, max_val)
    return max_val

In [11]:
def handle_solution(solver, goes_to, courier_assignment, num_couriers, num_items, distance_matrix, max_courier_distance):
    solver.set("timeout", 60)  # timeout is in milliseconds
    model = solver.model()
    route_matrix = [[model.evaluate(goes_to[i][j]) for j in range(num_items + 1)] for i in range(num_items + 1)]
    item_courier_matrix = [[model.evaluate(courier_assignment[i][k]) for k in range(num_couriers)] for i in range(num_items)]

    num_nodes = len(route_matrix) - 1
    routes = {}
    courier_distance = {}

    for k in range(num_couriers):
      id_items_to_transport = []
      for i in range(num_items):
          if item_courier_matrix[i][k]:
              id_items_to_transport.append(i+1)

      courier_path = []
      courier_total_distance = 0
      current_node = num_nodes

      courier_path.append(0)

      for item in id_items_to_transport:
          next_node = item - 1
          courier_path.append(next_node+1)
          courier_total_distance += distance_matrix[current_node][next_node]
          current_node = next_node

      courier_path.append(0)

      # Calculate the distance from the last node to the origin node
      courier_total_distance += distance_matrix[current_node][0]
      if len(courier_path)>2:
        routes[k+1] = courier_path
        courier_distance[k+1] = courier_total_distance

    total_distance=0

    # Print the courier paths and distances
    print("Courier Paths:")
    print("-" * 60)
    print("{:<10s} {:<30s} {:<15s}".format("Courier", "Path", "Distance"))
    print("-" * 60)
    for courier_id, route in routes.items():
        path_display = " > ".join([str(node) for node in route])
        distance_display = f"{courier_distance[courier_id]:,}"
        total_distance += courier_distance[courier_id]

        print("{:<10d} {:<30s} {:<15s}".format(courier_id, path_display, distance_display))
    print("-" * 60)

    print("Total distance:", f"{total_distance:,}")

    # Print the maximum distance travelled by any courier
    max_courier_distance_value = model.evaluate(max_courier_distance).as_long()
    print("Maximum distance travelled by any courier:", max_courier_distance_value)
    return max_courier_distance_value

In [12]:
def assign(num_couriers, num_items, max_loads, item_sizes, distance_matrix):
    if sum(item_sizes) > sum(max_loads):
        print("Total weight of all items surpasses possible weight transportable")
        return 0

    remaining_time = 300
    solver, goes_to, courier_assignment, max_courier_distance = define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix)

    time_before = time.time()
    if solver.check() == sat:
        total_distance = handle_solution(solver, goes_to, courier_assignment, num_couriers, num_items, distance_matrix, max_courier_distance)

        elapsed_time = time.time()-time_before
        remaining_time -= elapsed_time
        print(f"Initial solution found with total distance: {total_distance}")

        while remaining_time > 0:
            print(f"Looking for solutions with total distance < {total_distance}")
            time_before = time.time()
            solver, goes_to, courier_assignment, max_courier_distance = define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix, total_distance)

            if solver.check() == sat:
                total_distance = handle_solution(solver, goes_to, courier_assignment, num_couriers, num_items, distance_matrix)
                elapsed_time = time.time()-time_before
                remaining_time -= elapsed_time
                print(f"Elapsed time: {elapsed_time:.2f} seconds")
                print(f"Remaining time: {remaining_time:.2f} seconds")
            else:
                print("No more solutions found.")
                break

        if remaining_time <= 0:
            print("Time limit exceeded")
        return 0
    else:
        print("Failed to find a solution")
        return 0

In [13]:
data = data_list[6]
m = data['m']  # num_couriers
n = data['n']  # num_items
l = data['l']  # courier_loads
s = data['s']  # item_sizes
D = data['D']  # distance_matrix
assign(m,n,l,s,D)

Courier Paths:
------------------------------------------------------------
Courier    Path                           Distance       
------------------------------------------------------------
1          0 > 2 > 6 > 0                  239            
2          0 > 7 > 0                      179            
3          0 > 5 > 0                      119            
4          0 > 1 > 8 > 0                  370            
5          0 > 4 > 0                      141            
6          0 > 3 > 0                      141            
7          0 > 9 > 0                      87             
8          0 > 10 > 0                     133            
------------------------------------------------------------
Total distance: 1,409
Maximum distance travelled by any courier: 344
Initial solution found with total distance: 344
Looking for solutions with total distance < 344
No more solutions found.


0

In [14]:
from itertools import combinations

def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    return [Or(Not(pair[0]), Not(pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one_np(bool_vars):
    return at_most_one_np(bool_vars) + [at_least_one_np(bool_vars)]

In [15]:
def at_least_one_seq(bool_vars):
    clauses = []
    for i in range(1, len(bool_vars) + 1):
        clause = Or(bool_vars[:i])
        clauses.append(clause)
    return And(clauses)

def at_most_one_seq(bool_vars):
    clauses = []
    for pair in combinations(bool_vars, 2):
        clause = Or(Not(pair[0]), Not(pair[1]))
        clauses.append(clause)
    return And(clauses)

def exactly_one_seq(bool_vars):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars))